In [123]:
import os
import numpy as np
import cv2 as cv
import glob
from pathlib import Path
print(cv.__version__)

img_type = 'jpg'
calib_dir = "./my_imgs/pic3/calib_imgs"
rotate_dir = "./my_imgs/pic3/rotate_imgs/"

grid_size = 0.03  # 每个网格尺寸，单位米
grid_shape = (8, 6)  # 内角点列数、行数

4.6.0


In [124]:
# 相机标定
def calib(grid_shape, grid_size, img_dir, img_type):
    print(grid_shape)  # cv::Size(columns，rows)
    w, h = grid_shape

    # cp_int: 世界坐标系中int格式的角点序号坐标，如 (0,0,0), (1,0,0), (2,0,0) ....,(10,7,0).
    cp_int = np.zeros((w*h,3), np.float32)
    cp_int[:,0:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)
    # cp_world: 世界坐标系中的角点坐标
    cp_world = cp_int * grid_size

    obj_points = []  # 空间坐标系中的点
    img_points = []  # 像素坐标系中的点
    images = glob.glob(img_dir + os.sep + '**.' + img_type)  # 查找符合规则的文件路径名

    draw_save_dir = img_dir.replace('calib_imgs', 'calib_corners')
    if not Path(draw_save_dir).is_dir():
        os.mkdir(draw_save_dir)

    for i, fname in enumerate(images):
        img = cv.imread(fname)
        gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        # 查找角点，cp_img: 像素坐标系中的角点坐标
        ret, cp_img = cv.findChessboardCorners(gray_img, grid_shape, None)
        if ret == True:
            obj_points.append(cp_world)
            img_points.append(cp_img)
            draw_img = cv.drawChessboardCorners(img, grid_shape, cp_img, ret)
            cv.imwrite(os.path.join(draw_save_dir, str(i)+'.jpg'), draw_img)
        else:
            print(fname)
            raise RuntimeError("Find Chessboard Corners Error !")

    # 相机标定
    ret, K, coff_dis, v_rot, v_trans = cv.calibrateCamera(obj_points, img_points, gray_img.shape[::-1], None, None, flags=(cv.CALIB_ZERO_TANGENT_DIST+cv.CALIB_FIX_K3))
    print("Ret:", ret)
    print("Internal matrix:\n", K)
    print("Distortion Cofficients:\n", coff_dis)
    # print("Rotation vectors:\n", v_rot[0])
    # print("Translation vectors:\n", v_trans[0])
    # 计算重投影误差
    total_error = 0
    for i in range(len(obj_points)):
        # 世界坐标系中的点重投影到像素坐标系中
        img_points_repro, _ = cv.projectPoints(obj_points[i], v_rot[i], v_trans[i], K, coff_dis)
        error = cv.norm(img_points[i], img_points_repro, cv.NORM_L2) / len(img_points_repro)
        total_error += error
    print("Average Error of Reproject:", total_error/len(obj_points))

    return K, coff_dis

K, coff_dis = calib(grid_shape, grid_size, calib_dir, img_type);

(8, 6)
Ret: 0.4451177419944304
Internal matrix:
 [[1.15591174e+03 0.00000000e+00 8.15002368e+02]
 [0.00000000e+00 1.16058292e+03 6.13394808e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Cofficients:
 [[-0.12510674  0.16955632  0.          0.          0.        ]]
Average Error of Reproject: 0.05458250305059955


In [125]:
# PnP问题求解相机位姿
def usePnP(idx, K, coff_dis, grid_shape, grid_size, img_dir, img_type):
    w, h = grid_shape

    draw_save_dir = img_dir.replace('rotate_imgs', 'rotate_corners')
    if not Path(draw_save_dir).is_dir():
        os.mkdir(draw_save_dir)

    # cp_int: 世界坐标系中int格式的角点序号坐标，如 (0,0,0), (1,0,0), (2,0,0) ....,(10,7,0).
    cp_int = np.zeros((w*h,3), np.float32)
    cp_int[:,0:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)
    # cp_world: 世界坐标系中的角点坐标
    cp_world = cp_int * grid_size

    # cp_img: 像素坐标系中的角点坐标
    images = glob.glob(img_dir + os.sep + '**.' + img_type)  # 查找符合规则的文件路径名
    img = cv.imread(images[idx])
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, cp_img = cv.findChessboardCorners(gray_img, grid_shape, None)
    if ret == True:
        draw_img = cv.drawChessboardCorners(img, grid_shape, cp_img, ret)
        cv.imwrite(os.path.join(draw_save_dir, str(idx)+'.jpg'), draw_img)
    else:
        print(images[idx])
        raise RuntimeError("Find Chessboard Corners Error !")

    # 估计相机位姿
    _, v_rot, t = cv.solvePnP(cp_world, cp_img, K, coff_dis)
    # print("Rotation vectors:\n", v_rot)
    # print("Translation vectors:\n", t)

    # 计算累积重投影误差
    total_error = 0
    # 世界坐标系中的点重投影到像素坐标系中
    cp_repro, _ = cv.projectPoints(cp_world, v_rot, t, K, coff_dis)
    for i in range(len(cp_world)):
        error = cv.norm(cp_repro[i], cp_img[i], cv.NORM_L2) ** 2
        total_error += error
    total_error /= len(cp_world)
    print("Average Error of Reproject:", total_error)

    return v_rot, t

In [213]:
# 使用PnP求解世界坐标系到相机坐标系的变换矩阵
pic_num = 9  # 图像数目
vr_w2i = []  # 世界坐标系到相机坐标系i的旋转向量
t_w2i = []  # 世界坐标系到相机坐标系i的平移向量
R_w2i = []  # 世界坐标系到相机坐标系i的旋转矩阵

for i in range(pic_num):
    vr, t = usePnP(i, K, coff_dis, grid_shape, grid_size, rotate_dir, img_type)
    R = cv.Rodrigues(vr)[0]
    vr_w2i.append(vr)
    t_w2i.append(t)
    R_w2i.append(R)

Average Error of Reproject: 0.11538714752532542
Average Error of Reproject: 0.21571349288569763
Average Error of Reproject: 0.054349913417051234
Average Error of Reproject: 0.11989327710277091
Average Error of Reproject: 0.11948092320623498
Average Error of Reproject: 0.16818677119833106
Average Error of Reproject: 0.0559194041416049
Average Error of Reproject: 0.1134667625107492
Average Error of Reproject: 0.057673861583073936


In [214]:
# 计算第i帧到第i+1帧之间的位姿变换，求旋转向量表示
R_i2i1 = []  # 第i帧到第i+1帧的相机旋转矩阵
vr_i2i1 = []  # 第i帧到第i+1帧的相机旋转向量

for i in range(pic_num-1):
    R = R_w2i[i+1] @ R_w2i[i].T
    vr = cv.Rodrigues(R)[0]
    print(vr / np.linalg.norm(vr), np.linalg.norm(vr) * 180 / np.pi)
    R_i2i1.append(R)
    vr_i2i1.append(vr)

[[-0.00814482]
 [-0.99995124]
 [-0.00558429]] 30.891424043763905
[[-1.96310944e-02]
 [ 9.99806999e-01]
 [-7.64563235e-04]] 31.713003468803056
[[ 0.01493871]
 [-0.99950915]
 [-0.02753708]] 29.05328826323063
[[0.00554881]
 [0.99942986]
 [0.03330416]] 28.420992809178717
[[-0.01559423]
 [-0.99925627]
 [-0.03526664]] 28.284047695384785
[[-0.00416111]
 [ 0.99968131]
 [ 0.0248989 ]] 29.2774963462251
[[-0.01170588]
 [-0.99974062]
 [-0.01953617]] 29.583920998953186
[[-0.00270293]
 [ 0.99977375]
 [ 0.02109827]] 29.499473909192474


In [215]:
600 * 0.0514

30.84

In [216]:
# 计算第0帧到第i帧之间的位姿变换，求旋转向量表示
R_02i = []  # 第0帧到第i帧的相机旋转矩阵
vr_02i = []  # 第0帧到第i帧的相机旋转向量

for i in range(1, pic_num):
    R = R_w2i[i] @ R_w2i[0].T
    vr = cv.Rodrigues(R)[0]
    print(vr / np.linalg.norm(vr), np.linalg.norm(vr) * 180 / np.pi)
    R_02i.append(R)
    vr_02i.append(vr)

[[-0.00814482]
 [-0.99995124]
 [-0.00558429]] 30.891424043763905
[[-0.73435473]
 [ 0.67775613]
 [ 0.03701014]] 1.2044630203366211
[[-0.01546937]
 [-0.99928166]
 [-0.03459567]] 28.23610847392416
[[-0.82343248]
 [ 0.55529029]
 [ 0.11666897]] 0.3359986860822683
[[-0.02562268]
 [-0.99900228]
 [-0.03657758]] 28.101922487998227
[[-0.59831017]
 [ 0.79963951]
 [-0.05100583]] 1.4855872538106036
[[-0.04182232]
 [-0.9986406 ]
 [-0.03111014]] 28.422366124661906
[[-0.75908926]
 [ 0.64970534]
 [ 0.04082242]] 1.6942963233656672


In [217]:
# 计算第i帧到第0帧之间的位姿变换，求旋转向量表示
R_i20 = []  # 第i帧到第0帧的相机旋转矩阵
vr_i20 = []  # 第i帧到第0帧的相机旋转向量

for i in range(1, pic_num):
    R = R_w2i[0] @ R_w2i[i].T
    vr = cv.Rodrigues(R)[0]
    print(vr / np.linalg.norm(vr), np.linalg.norm(vr) * 180 / np.pi)
    R_i20.append(R)
    vr_i20.append(vr)

[[0.00814482]
 [0.99995124]
 [0.00558429]] 30.891424043763905
[[ 0.73435473]
 [-0.67775613]
 [-0.03701014]] 1.2044630203366211
[[0.01546937]
 [0.99928166]
 [0.03459567]] 28.23610847392416
[[ 0.82343248]
 [-0.55529029]
 [-0.11666897]] 0.3359986860822683
[[0.02562268]
 [0.99900228]
 [0.03657758]] 28.101922487998227
[[ 0.59831017]
 [-0.79963951]
 [ 0.05100583]] 1.485587253809622
[[0.04182232]
 [0.9986406 ]
 [0.03111014]] 28.422366124661934
[[ 0.75908926]
 [-0.64970534]
 [-0.04082242]] 1.6942963233656672
